In [42]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import io

from scipy import misc

from sklearn.metrics import accuracy_score

%matplotlib inline


In [43]:
cid ="fd0f694eb3df41f1ae48bc7d0d8d5a3a" 
secret = "f06d7ba6bc094ea5944622f77c77462d"
username = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope,cid,secret, redirect_uri='http://localhost:8888')
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [44]:
#Top Songs 2019 - Sahil Morchi
good_playlist = sp.user_playlist("tCdK3fA0QheTckL2vXPe5A", "37i9dQZF1Etr76TQzqcYgZ")
#Top Songs 2017 - Sahil Morchi
bad_playlist = sp.user_playlist("xgsMa8P0T-yZdSqmrLtbUA", "37i9dQZF1E9HVsIaNueEEn")

#Goal: Analyze trend in listening habits over time

In [45]:

good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"]
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
print(len(good_songs))
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])
good_tracks



100


{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1Etr76TQzqcYgZ/tracks?offset=0&limit=100',
 'items': [{'added_at': '2019-12-12T21:06:23Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3hyEbRtp617pNCuuQjyOmc'},
       'href': 'https://api.spotify.com/v1/artists/3hyEbRtp617pNCuuQjyOmc',
       'id': '3hyEbRtp617pNCuuQjyOmc',
       'name': 'Lost Kings',
       'type': 'artist',
       'uri': 'spotify:artist:3hyEbRtp617pNCuuQjyOmc'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE'

In [46]:
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"]
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
print(len(bad_songs))
for i in range(len(bad_songs)- 500):
    bad_ids.append(bad_songs[i]['track']['id'])
bad_tracks

100


{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1E9HVsIaNueEEn/tracks?offset=0&limit=100',
 'items': [{'added_at': '2017-11-18T01:43:51Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7mDU6nMUJnOSY2Hkjz5oqM'},
       'href': 'https://api.spotify.com/v1/artists/7mDU6nMUJnOSY2Hkjz5oqM',
       'id': '7mDU6nMUJnOSY2Hkjz5oqM',
       'name': 'Roy Woods',
       'type': 'artist',
       'uri': 'spotify:artist:7mDU6nMUJnOSY2Hkjz5oqM'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',


In [47]:
#get the audio features for the good playlist
features = []
inSavedTracks = []
j = 0
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 1
j = 0
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs[j]
        j= j+1
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['artistPopularity'] = sp.artist(track['track']['artists'][0]['id'])['popularity']
        features[-1]['target'] = 0

In [48]:
trainingData = pd.DataFrame(features)
trainingData.head()

""


In [49]:
red_blue = ['#19B5FE', '#EF4836']
palette = sns.color_palette(red_blue)
sns.set_palette(palette)
sns.set_style('white')

In [52]:
train, test = train_test_split(trainingData, test_size = 0.15, train_size = 0.85)
print("Training size: {}, Test size: {}".format(len(train),len(test)))

pos_tempo = trainingData[trainingData['target'] == 1]['tempo']
neg_tempo = trainingData[trainingData['target'] == 0]['tempo']
pos_dance = trainingData[trainingData['target'] == 1]['danceability']
neg_dance = trainingData[trainingData['target'] == 0]['danceability']
pos_duration = trainingData[trainingData['target'] == 1]['duration_ms']
neg_duration = trainingData[trainingData['target'] == 0]['duration_ms']
pos_loudness = trainingData[trainingData['target'] == 1]['loudness']
neg_loudness = trainingData[trainingData['target'] == 0]['loudness']
pos_speechiness = trainingData[trainingData['target'] == 1]['speechiness']
neg_speechiness = trainingData[trainingData['target'] == 0]['speechiness']
pos_valence = trainingData[trainingData['target'] == 1]['valence']
neg_valence = trainingData[trainingData['target'] == 0]['valence']
pos_energy = trainingData[trainingData['target'] == 1]['energy']
neg_energy = trainingData[trainingData['target'] == 0]['energy']
pos_acousticness = trainingData[trainingData['target'] == 1]['acousticness']
neg_acousticness = trainingData[trainingData['target'] == 0]['acousticness']
pos_key = trainingData[trainingData['target'] == 1]['key']
neg_key = trainingData[trainingData['target'] == 0]['key']
pos_instrumentalness = trainingData[trainingData['target'] == 1]['instrumentalness']
neg_instrumentalness = trainingData[trainingData['target'] == 0]['instrumentalness']
pos_popularity = trainingData[trainingData['target'] == 1]['trackPopularity']
neg_popularity = trainingData[trainingData['target'] == 0]['trackPopularity']

ValueError: With n_samples=0, test_size=0.15 and train_size=0.85, the resulting train set will be empty. Adjust any of the aforementioned parameters.